In [3]:
import matplotlib as mpl
import mpl_toolkits.axisartist as AA
from matplotlib import colors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
import time

# Python code for Julia Fractal 
from PIL import Image 

import random
import array
import math
import numpy as np
import pandas as pd
import datashader as ds
from datashader import transfer_functions as tf
from datashader.colors import inferno, viridis
from numba import jit
from math import sin, cos, sqrt, fabs

ModuleNotFoundError: No module named 'datashader'

# <center>Definition</center>
<center>xn+1 = sin(a yn) + c cos(a xn)</center>
<center>yn+1 = sin(b xn) + d cos(b yn)</center>
<center>where a, b, c, d are variables that define each attractor.</center>

> If I substitute the follow numbers in for the valiables, this is what I get.
> firgure 1 -  a = -1.4, b = 1.6, c = 1.0, d = 0.7
> firgure 2 -  a = 1.6, b = -0.6, c = -1.2, d = 1.6 
> figure 3 -  a = 1.7, b = 1.7, c = 0.6, d = 1.2 
> figure 5 - a = 1.5, b = -1.8, c = 1.6, d = 0.9
> figure 6 - a = -1.7, b = 1.3, c = -0.1, d = -1.2 
> figure 7 - a = -1.7, b = 1.8, c = -1.9, d = -0.4 
> figure 8 - a = -1.8, b = -2.0, c = -0.5, d = -0.9 


Question: How are the color effects here achieved? 
Answer: The main thing happening here is that I don't draw the attractor to the final image. Rather I create a large grid of 32 bit (int or float) and instead of drawing into that in color I evaluate points on the attractor and just increment each cell of the grid if the attractor passes through it. So it's essentially a 2D histogram for occupancy. One wants to evaluate the attractor much more/longer than normal in order to create a reasonable dynamic range and ultimately smooth color gradients. I then save this 2D grid, the process of applying smooth color gradients comes as a secondary process ... better than trying to encode the right color during the generation process. One can even just save the grid as a 16 or 32 bit raw, open in PhotoShop and apply custom gradient maps there. 
Of course this is "just" a density mapping of the histogram and doesn't immediately allow for coloring based upon other attributes of the attractor path, such as curvature. But such attributes can be encoded into the histogram encoding, for example the amount added to a cell being a function of curvature.

In [4]:
x\u(n+1) = math.sin(a y\un) + c math.cos(a x\un)
y\u(n+1) = math.sin(b x\un) + d math.cos(b y\un)

SyntaxError: unexpected character after line continuation character (<ipython-input-4-1abc282781ad>, line 1)

In [17]:
Tachyon
The images here were rendered using the Tachyon rendering package written by John Stone. The same results could be achieved with almost any raytracer, in particular PovRay, Radiance, etc. As mentioned before it is important to be able to set the ray trace depth, it is also important to be able to precisely set the position and radius of the balls to avoid any narrow slits or intersections. The table below gives the centre and radius of the 4 balls in the configuration used here.

      x      y      z         r
    -0.5   -0.5    0.5   sqrt(2) / 2 
     0.5    0.5    0.5   sqrt(2) / 2 
    -0.5    0.5   -0.5   sqrt(2) / 2 
     0.5   -0.5   -0.5   sqrt(2) / 2


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 5)

In [18]:
def Clifford(x, y, a, b, c, d, *o):
    return math.sin(a * y) + c * math.cos(a * x), \
           math.sin(b * x) + d * math.cos(b * y)

n=10000000


def trajectory(fn, x0, y0, a, b=0, c=0, d=0, e=0, f=0, n=n):
    x, y = np.zeros(n), np.zeros(n)
    x[0], y[0] = x0, y0
    for i in np.arange(n-1):
        x[i+1], y[i+1] = fn(x[i], y[i], a, b, c, d, e, f)
    return pd.DataFrame(dict(x=x,y=y))

%%time
df = trajectory(Clifford, 0, 0, -1.3, -1.3, -1.8, -1.9)

CPU times: user 25.5 s, sys: 400 ms, total: 25.9 s
Wall time: 26.4 s


In [21]:
df.tail()

,x,y
9999995,0.798129,0.098356
9999996,-1.042485,-2.745660
9999997,-0.799859,2.705656
9999998,-0.544501,2.629746
9999999,-1.094023,2.477735


We can now aggregate these 10,000,000 continuous coordinates into a discrete 2D rectangular grid with Datashader, counting each time a point fell into that grid cell: 

In [22]:
%%time

cvs = ds.Canvas(plot_width = 700, plot_height = 700)
agg = cvs.points(df, 'x', 'y')
print(agg.values[190:195,190:195],"\n")
[[ 44  33  33  29  32]
 [ 34  34  44  33  39]
 [127  39  33  38  33]
 [157 199 114  57  57]
 [ 84  97 136 123 108]]

SyntaxError: invalid syntax (<unknown>, line 5)

A small portion of that grid is shown above, but it's difficult to see the grid's structure from the numerical values. To see the entire array at once, we can turn each grid cell into a pixel, using a greyscale value from white to black: As you can see, the most-visited areas of the plane have an interesting structure for this set of parameters. To explore further, let's wrap up the above aggregation and shading commands into a function so we can apply them more easily:

NameError: name 'ds' is not defined

In [24]:
%%time

cvs = ds.Canvas(plot_width = 700, plot_height = 700)
agg = cvs.points(df, 'x', 'y')
print(agg.values[190:195,190:195],"\n")
[[ 44  33  33  29  32]
 [ 34  34  44  33  39]
 [127  39  33  38  33]
 [157 199 114  57  57]
 [ 84  97 136 123 108]]

ds.transfer_functions.Image.border=0

tf.shade(agg, cmap = ["white", "black"])

def dsplot(fn, vals, n=n, cmap=viridis, label=True):
    """Return a Datashader image by collecting `n` trajectory points for the given attractor `fn`"""
    lab = ("{}, "*(len(vals)-1)+" {}").format(*vals) if label else None
    df  = trajectory(fn, *vals, n=n)
    cvs = ds.Canvas(plot_width = 300, plot_height = 300)
    agg = cvs.points(df, 'x', 'y')
    img = tf.shade(agg, cmap=cmap, name=lab)
    return img


NameError: name 'viridis' is not defined

In [ ]:
# And let's load some colormaps that we can use for subsequent plots:

from colorcet import palette
palette["viridis"]=viridis
palette["inferno"]=inferno

# We can now use these colormaps with a pre-selected set of Clifford attractor parameter values (stored in a separate YAML-format text file) to show a wide variety of trajectories that these equations can form:

import yaml
vals = yaml.load(open("attractors.yml","r"))

def args(name):
    """Return a list of available argument lists for the given type of attractor"""
    return [v[1:] for v in vals if v[0]==name]

def plot(fn, vals=None, **kw):
    """Plot the given attractor `fn` once per provided set of arguments."""
    vargs=args(fn.__name__) if vals is None else vals
    return tf.Images(*[dsplot(fn, v[1:], cmap=palette[v[0]][::-1], **kw) for v in vargs]).cols(4)
plot(Clifford)

Here the values shown are the arguments for the first call to Clifford(x, y, a, b, c, d), with each subsequent call using the x,y location of the previous call.

Randomly sampling the parameter space typically yields much less dramatic patterns, such as all trajectory locations being on a small number of points:

In [ ]:
import numpy.random
numpy.random.seed(21)
num = 4

rvals=np.c_[np.zeros((num,2)), numpy.random.random((num,4))*4-2]
plot(Clifford, vals=[["kbc"]+list(rvals[i]) for i in range(len(rvals))], label=True)

If you wish, Datashader could easily be used to filter out such uninteresting examples, by applying a criterion to the aggregate array before shading and showing only those that remain (e.g. rejecting those where 80% of the pixel bins are empty).

De Jong attractors
A variety of other sets of attractor equations have been proposed, such as these from Peter de Jong:

In [ ]:
@jit
def De_Jong(x, y, a, b, c, d, *o):
    return sin(a * y) - cos(b * x), \
           sin(c * x) - cos(d * y)

plot(De_Jong)

Svensson attractors
From Johnny Svensson:

In [ ]:
@jit
def Svensson(x, y, a, b, c, d, *o):
    return d * sin(a * x) - sin(b * y), \
           c * cos(a * x) + cos(b * y)

plot(Svensson)

Bedhead Attractor
From Ivan Emrich and Jason Rampe:

In [ ]:
@jit
def Bedhead(x, y, a, b, *o):
    return sin(x*y/b)*y + cos(a*x-y), \
           x + sin(y)/b

plot(Bedhead)

Fractal Dream Attractor
From Clifford A. Pickover's book “Chaos In Wonderland”, with parameters from Jason Rampe:

In [ ]:
@jit
def Fractal_Dream(x, y, a, b, c, d, *o):
    return sin(y*b)+c*sin(x*b), \
           sin(x*a)+d*sin(y*a)

plot(Fractal_Dream)

In [ ]:
Hopalong attractors
From Barry Martin, here with code for two variants from François Pacull:

In [ ]:
@jit
def Hopalong1(x, y, a, b, c, *o):
    return y - sqrt(fabs(b * x - c)) * np.sign(x), \
           a - x
@jit
def Hopalong2(x, y, a, b, c, *o):
    return y - 1.0 - sqrt(fabs(b * x - 1.0 - c)) * np.sign(x - 1.0), \
           a - x - 1.0

plot(Hopalong1)

Gumowski-Mira Attractor
From I. Gumowski and C. Mira, with code and parameters from Jason Rampe and Lázaro Alonso:

In [ ]:
@jit
def G(x, mu):
    return mu * x + 2 * (1 - mu) * x**2 / (1.0 + x**2)

@jit
def Gumowski_Mira(x, y, a, b, mu, *o):
    xn = y + a*(1 - b*y**2)*y  +  G(x, mu)
    yn = -x + G(xn, mu)
    return xn, yn

plot(Gumowski_Mira)

Symmetric Icon Attractor
The Hopalong and Gumowski-Mira equations often result in symmetric patterns, but a different approach is to force the patterns to be symmetric, which is often pleasing. Examples from “Symmetry in Chaos” by Michael Field and Martin Golubitsky, with code and parameters from Jason Rampe:


In [ ]:
@jit
def Symmetric_Icon(x, y, a, b, g, om, l, d, *o):
    zzbar = x*x + y*y
    p = a*zzbar + l
    zreal, zimag = x, y

    for i in range(1, d-1):
        za, zb = zreal * x - zimag * y, zimag * x + zreal * y
        zreal, zimag = za, zb

    zn = x*zreal - y*zimag
    p += b*zn

    return p*x + g*zreal - om*y, \
           p*y - g*zimag + om*x

plot(Symmetric_Icon)

Interactive plotting
If you are running a live Python process, you can use Datashader with HoloViews and Bokeh to zoom in and see the individual steps in any of these calculations:

In [ ]:
import holoviews as hv
from holoviews.operation.datashader import datashade, dynspread
hv.extension('bokeh')

dynspread(datashade(hv.Points(trajectory(Clifford, *(args("Clifford")[5][1:]))),
                    cmap=viridis[::-1]).opts(width=400,height=400))

In [ ]:
Each time you zoom in in a live process, the data will be reaggregated, which should take a small fraction of a second for 10 million points. Eventually, once you zoom in enough you should see individual data points, as we are not connecting the points into a trajectory here.

You can also try "connecting the dots", which will reveal how the particle jumps discretely from one region of the space to another:


In [ ]:
dynspread(datashade(hv.Path([trajectory(Clifford, *(args("Clifford")[5][1:]))]),
                    cmap=viridis[::-1]).opts(width=400,height=400))

Again, if you zoom in on a live server, the plot will update so that you can see the individual traces involved.

On the live server, you can also explore to find your own parameter values that generate interesting patterns:

In [ ]:
def hv_clif(a,b,c,d,x0=0,y0=0,n=n):
    return datashade(hv.Points(trajectory(Clifford, x0, y0, a, b, c, d, n)),
                     cmap=inferno[::-1], dynamic=False)

x0,y0,a,b,c,d = args("Clifford")[6][1:]

dm = hv.DynamicMap(hv_clif, kdims=['a', 'b', 'c', 'd'])
dm = dm.redim.range(a=(-2.0, 2.0), b=(-2.0,2.0), c=(-2.0,2.0), d=(-2.0,2.0))
dm = dm.redim.default(a=a, b=b, c=c, d=d).opts(width=500,height=500)
dm

Although many of the regions of this four-dimensional parameter space generate uninteresting trajectories such as single points, you can find interesting regions by starting with one of the a,b,c,d tuples of values in previous plots, then click on one slider and use the left and right arrow keys to see how the plot changes as that parameter changes. See also this Panel-based attractor dashboard.